In [ ]:
# === Automatyczna instalacja bibliotek wraz z wymaganymi rozszerzeniami PyG ===
import sys

# Dostosuj ten link do swojej wersji PyTorch i CUDA:
# CUDA 11.7 (z GPU)
torch_geometric_whl = "https://data.pyg.org/whl/torch-2.0.1+cu117.html"

# Jeśli masz CPU-only, odkomentuj tę linię zamiast powyższej:
# torch_geometric_whl = "https://data.pyg.org/whl/torch-2.0.1+cpu.html"

!{sys.executable} -m pip install -q ase torch tqdm requests
!{sys.executable} -m pip install -q torch-geometric -f {torch_geometric_whl}
!{sys.executable} -m pip install -q torch-scatter torch-sparse torch-cluster torch-spline-conv -f {torch_geometric_whl}

# === Importy ===
import os
import requests
from io import StringIO
from ase.io import read
import numpy as np
import torch
from torch_geometric.data import Data
from torch_geometric.nn import radius_graph
from tqdm import tqdm  # <=== UWAGA: używamy wersji konsolowej tqdm

# === Parametry ===
save_dir = "graph_dataset"
os.makedirs(save_dir, exist_ok=True)
perturbations_per_cif = 3
noise_magnitude = 0.6

def download_cif(url):
    response = requests.get(url)
    if response.status_code == 200:
        return StringIO(response.text)
    else:
        print(f"⛔ Błąd pobierania: {url}")
        return None

def make_graph_from_atoms(atoms, radius=5.0):
    pos = torch.tensor(atoms.get_positions(), dtype=torch.float)
    z = torch.tensor(atoms.get_atomic_numbers(), dtype=torch.long)
    edge_index = radius_graph(pos, r=radius)
    return Data(x=z.view(-1, 1), pos=pos, edge_index=edge_index)

def perturb_coordinates(atoms, magnitude=0.3):
    new_atoms = atoms.copy()
    noise = np.random.normal(0, magnitude, new_atoms.positions.shape)
    new_atoms.set_positions(new_atoms.positions + noise)
    return new_atoms

# === Wczytaj linki z pliku COD-selection.txt ===
try:
    with open("COD-selection.txt", "r") as f:
        cif_links = [line.strip() for line in f if line.strip()]
    if not cif_links:
        print("⛔ Plik 'COD-selection.txt' jest pusty.")
    else:
        print(f"🔍 Znaleziono {len(cif_links)} linków CIF do przetworzenia.")
except FileNotFoundError:
    print("⛔ Nie znaleziono pliku 'COD-selection.txt'. Upewnij się, że znajduje się w tym samym folderze.")
    cif_links = []

# === Przerwij, jeśli brak CIF-ów ===
if not cif_links:
    print("⚠️  Brak danych do przetworzenia. Zakończono.")
else:
    # === Przetwarzanie CIF-ów z paskiem postępu i licznikami ===
    success = 0
    fail = 0

    for i, url in enumerate(tqdm(cif_links, desc="⏳ Przetwarzanie CIF-ów")):
        print(f"\n📥 [{i+1}/{len(cif_links)}] Pobieram: {url}")

        cif_stream = download_cif(url)
        if cif_stream is None:
            fail += 1
            continue

        try:
            atoms = read(cif_stream, format='cif')
            base_name = os.path.basename(url).replace('.cif', '')

            # Zapisz dobrą konformację
            good_graph = make_graph_from_atoms(atoms)
            good_graph.y = torch.tensor([1])  # etykieta: dobra
            torch.save(good_graph, os.path.join(save_dir, f"{base_name}_good.pt"))

            # Zapisz błędne konformacje
            for j in range(perturbations_per_cif):
                bad_atoms = perturb_coordinates(atoms, magnitude=noise_magnitude)
                bad_graph = make_graph_from_atoms(bad_atoms)
                bad_graph.y = torch.tensor([0])  # etykieta: zła
                torch.save(bad_graph, os.path.join(save_dir, f"{base_name}_bad_{j+1}.pt"))

            success += 1
            print(f"✅ Zapisano grafy dla: {base_name}")

        except Exception as e:
            print(f"⚠️ Błąd przetwarzania {url}: {e}")
            fail += 1

    print(f"\n📊 Podsumowanie:")
    print(f"✅ Udało się przetworzyć: {success} struktur")
    print(f"❌ Błędy: {fail}")